In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import random
import os
import glob
import cv2
import gc
import keras
import tensorflow as tf

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from sklearn.metrics import roc_auc_score
from keras.layers import Dense, Activation, Dropout, Flatten 
import matplotlib.pyplot as plt
from tqdm import tqdm, trange
%matplotlib inline

base = '/kaggle/input/histopathologic-cancer-detection/'
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

Using TensorFlow backend.


In [2]:
labels = pd.read_csv(base+'train_labels.csv')
print(labels.head(10))
print(len(labels))

def load_data(idx) : 
    N = len(idx)
    X = np.empty([N, 96, 96, 3], dtype = np.uint8)
    y = np.squeeze([labels['label'][i] for i in idx])
    for i in range(N) : 
        X[i] = cv2.imread(base+'train/{}.tif'.format(labels['id'][idx[i]]))
    return X, y

                                         id  label
0  f38a6374c348f90b587e046aac6079959adf3835      0
1  c18f2d887b7ae4f6742ee445113fa1aef383ed77      1
2  755db6279dae599ebb4d39a9123cce439965282d      0
3  bc3f0c64fb968ff4a8bd33af6971ecae77c75e08      0
4  068aba587a4950175d04c680d38943fd488d6a9d      0
5  acfe80838488fae3c89bd21ade75be5c34e66be7      0
6  a24ce148f6ffa7ef8eefb4efb12ebffe8dd700da      1
7  7f6ccae485af121e0b6ee733022e226ee6b0c65f      1
8  559e55a64c9ba828f700e948f6886f4cea919261      0
9  8eaaa7a400aa79d36c2440a4aa101cc14256cda4      0
220025


In [5]:
N = len(labels)
idx = np.arange(len(labels))
np.random.seed(42)
np.random.shuffle(idx)
idx_cut = idx[0:N]
X, y = load_data(idx_cut)
print(X.shape, y.shape)

(220025, 96, 96, 3) (220025,)


Here's some exploratory analysis.

In [ ]:
positives = y == 1
fig = plt.figure(figsize=(10, 4), dpi=150)
np.random.seed(100) #we can use the seed to get a different set of random images
for plotNr,idx in enumerate(np.random.randint(0,N,8)):
    ax = fig.add_subplot(2, 8//2, plotNr+1, xticks=[], yticks=[]) #add subplots
    plt.imshow(X[idx]) #plot image
    ax.set_title('Label: ' + str(y[idx])) #show the label corresponding to the image
nbins = 256
colordic = {0 : 'R', 1 : 'G', 2 : 'B'}
fig, axs = plt.subplots(3, 2, figsize = (8, 8), sharey = True, dpi = 150)
plt.subplots_adjust(hspace = 0.5)
for i in range(3) : 
    i2 = 2*i
    ax = axs[i, 0]
    ax.hist(X[positives, :, :, i].flatten(), bins = nbins, density = True)
    ax.set_title('{} channel Positives'.format(colordic[i]), color = 'white')
    ax.tick_params(axis = 'both', colors = 'white')
    ax = axs[i, 1]
    ax.hist(X[~positives, :, :, i].flatten(), bins = nbins, density = True)
    ax.set_title('{} channel Negatives'.format(colordic[i]), color = 'white')
    ax.tick_params(axis = 'both', colors = 'white')
nbins = 64
fig, axs = plt.subplots(1, 2, sharey = True, figsize = (8, 2), dpi = 150)
axs[0].hist(np.mean(X[positives], axis = (1, 2, 3)).flatten(), bins = nbins, density = True)
axs[0].set_title('Mean Brightness of Positives', color = 'white')
axs[0].tick_params(axis = 'both', colors = 'white')
axs[1].hist(np.mean(X[~positives], axis = (1, 2, 3)).flatten(), bins = nbins, density = True)
axs[1].set_title('Mean Brightness of Negatives', color = 'white')
axs[1].tick_params(axis = 'both', colors = 'white')

In [6]:
filters = [3, 3, 3, 3, 3, 3, 3]
strid = [1, 1, 1]
pool = [2, 2, 2, 2, 2, 2, 2]
chan = [8, 16, 32, 64, 128, 256, 512, 1024]
data_for = 'channels_last'
model = Sequential()

i = 1
model.add(Conv2D(chan[i], (filters[i], filters[i]), padding = 'same', input_shape = [96, 96, 3]))
model.add(BatchNormalization(axis = -1))
model.add(Activation('relu'))
model.add(Conv2D(chan[i], (filters[i], filters[i]), padding = 'same'))
model.add(BatchNormalization(axis = -1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (pool[i], pool[i])))
model.add(Dropout(0.25))

for i in range(2, 5) : 
    model.add(Conv2D(chan[i], (filters[i], filters[i]), padding = 'same'))
    model.add(BatchNormalization(axis = -1))
    model.add(Activation('relu'))
    model.add(Conv2D(chan[i], (filters[i], filters[i]), padding = 'same'))
    model.add(BatchNormalization(axis = -1))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size = (pool[i], pool[i])))
    model.add(Dropout(0.25))
          
model.add(Flatten())
model.add(Dense(chan[-3]))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.summary()
model.compile(loss = 'binary_crossentropy', 
              optimizer = keras.optimizers.Adam(learning_rate = 0.0007), 
              metrics=['accuracy', tf.keras.metrics.AUC(curve = 'PR'), tf.keras.metrics.AUC(curve = 'ROC')])          

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 96, 96, 16)        448       
_________________________________________________________________
batch_normalization_1 (Batch (None, 96, 96, 16)        64        
_________________________________________________________________
activation_1 (Activation)    (None, 96, 96, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 96, 96, 16)        2320      
_________________________________________________________________
batch_normalization_2 (Batch (None, 96, 96, 16)        64        
_________________________________________________________________
activation_2 (Activation)    (None, 96, 96, 16)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 48, 48, 16)       

We will now take off some indices as a validation set. The parameter 0.2 represents the fraction of data to treat as validation set. The strategy is to generate nvals random numbers between 0 and total size of data, and use those as indices for validation data. We subtract these indices from a list from 0 to total size to get indices for the training data.

In [7]:
nvals = int(np.floor(0.2 * len(X)))
np.random.seed(1)
valid_indx = np.random.randint(0, len(X), nvals)
train_indx = np.delete(np.arange(len(X)), valid_indx)

epoch_num = 10
batch_size = 50
num_batch = int(np.floor(len(train_indx)/batch_size))

x_batch = np.empty([batch_size, 96, 96, 3], dtype = np.uint8)
y_batch = np.empty([batch_size])
for i in range(epoch_num) : 
    np.random.shuffle(train_indx)           
    with trange(num_batch) as t : 
        for j in t : 
            start = j * batch_size
            indxes = train_indx[start : start + batch_size]
            x_batch = X[indxes]
            y_batch = y[indxes]
            metrics = model.train_on_batch(x_batch, y_batch, reset_metrics = False)    
            t.set_description('Running training epoch ' + str(i)) #set progressbar title
            t.set_postfix(loss="%.2f" % round(metrics[0], 2), 
                          acc="%.2f" % round(metrics[1], 2), 
                          auc_pr = "%.2f" % round(metrics[2], 2), 
                          auc_roc = "%.2f" % round(metrics[3], 2)) #display metrics

Running training epoch 9: 100%|██████████| 3600/3600 [02:50<00:00, 21.09it/s, acc=0.92, auc_pr=0.93, auc_roc=0.95, loss=0.24]


In [ ]:
model.fit(x = X_train, y = y_train, batch_size = 50, epochs = 10, verbose = 1, validation_data=(X_valid, y_valid), shuffle = True)

In [ ]:
batch_size = 50
num_batch = int(np.floor(nvals/batch_size))

with trange(num_batch) as t : 
    for j in t : 
        start = j * batch_size
        indxes = valid_indx[start : start + batch_size]
        x_batch = X[indxes]
        y_batch = y[indxes]
        metrics = model.test_on_batch(x_batch, y_batch, reset_metrics = False)    
        t.set_description('Running validation epoch ') #set progressbar title
        t.set_postfix(loss="%.2f" % round(metrics[0], 2),
                      acc="%.2f" % round(metrics[1], 2),
                     auc = "%.2f" % round(metrics[2], 2)) #display metrics

1. Dense layer with same number of output neurons as last CNN layer seems to hamper learning - accuracy and loss stay more or less constant in this case. With a 4x larger layer we find a similar pattern - now accuracy increases but really slowly. The 2x pattern seems to hit a sweet spot. WHY?! Could be because our eyes probably perceive regions on log scale? Well brightness they do...
2. There's a sweet spot for number of parameters and actual architecture. Starting with 16 and ending at 256 for Conv layers works best.
3. Adding BatchNorm pushes accuracy to 0.83 in first epoch! Similar spike for PR and ROC AUC. Overall much better model with BatchNorm.

In [16]:
base_test_dir = base + 'test/' #specify test data folder
test_files = glob.glob(base_test_dir+'*.tif') #find the test file names
submission = pd.DataFrame() #create a dataframe to hold results
file_batch = 5000 #we will predict 5000 images at a time
max_idx = len(test_files) #last index to use
for idx in range(0, max_idx, file_batch): #iterate over test image batches
    print("Indexes: %i - %i"%(idx, idx+file_batch))
    test_df = pd.DataFrame({'path': test_files[idx:idx+file_batch]}) #add the filenames to the dataframe
    test_df['id'] = test_df.path.map(lambda x: x.split('/')[-1].split(".")[0]) #add the ids to the dataframe
    test_df['image'] = test_df['path'].map(cv2.imread) #read the batch
    K_test = np.stack(test_df["image"].values) #convert to numpy array
    predictions = model.predict(K_test,verbose = 1) #predict the labels for the test data
    test_df['label'] = predictions #store them in the dataframe
    submission = pd.concat([submission, test_df[["id", "label"]]])
submission.head() #display first lines

Indexes: 0 - 5000
5000/5000 [==============================] - 1s 295us/step
Indexes: 5000 - 10000
5000/5000 [==============================] - 1s 293us/step
Indexes: 10000 - 15000
5000/5000 [==============================] - 1s 291us/step
Indexes: 15000 - 20000
5000/5000 [==============================] - 1s 293us/step
Indexes: 20000 - 25000
5000/5000 [==============================] - 2s 300us/step
Indexes: 25000 - 30000
5000/5000 [==============================] - 1s 296us/step
Indexes: 30000 - 35000
5000/5000 [==============================] - 1s 294us/step
Indexes: 35000 - 40000
5000/5000 [==============================] - 1s 297us/step
Indexes: 40000 - 45000
5000/5000 [==============================] - 1s 291us/step
Indexes: 45000 - 50000
5000/5000 [==============================] - 1s 291us/step
Indexes: 50000 - 55000
5000/5000 [==============================] - 1s 298us/step
Indexes: 55000 - 60000
2458/2458 [==============================] - 1s 350us/step


,id,label
0,dc308167f29b617cac141c34a7643a4fb86eb3ff,0.927498
1,68084861d7deeeeaf1669068a09f69aecfb51709,0.941825
2,b693e5fe5432918c7924088449957e7e33fb3121,0.047461
3,a6d81cc35bd175ef955d794b6d948ee728e4be13,0.007930
4,f1e6671e498b7a8680812b8b1296cda5a9240975,0.204908


In [17]:
submission.to_csv("submission.csv", index = False, header = True)